# Segmenting remote sensing imagery with point prompts

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_point_prompts.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/sam2_point_prompts.ipynb)

This notebook shows how to generate object masks from point prompts with the Segment Anything Model 2 (SAM 2). 

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
%pip install -U segment-geospatial

## Import libraries

In [1]:
import leafmap
from samgeo import SamGeo2, regularize

## Create an interactive map

In [7]:
m = leafmap.Map(center=[24.653287, 41.588070], zoom=16, height="800px")
m.add_basemap("Satellite")
m

Map(center=[24.653287, 41.58807], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map. If no geometry is drawn, the default bounding box will be used.

You can also use your own image. Uncomment and run the following cell to use your own image.

In [3]:
image = r'D:\Git\Geo_AI\data\Sam_in_colab\SAM_experiment.tif'

In [4]:
image

'D:\\Git\\Geo_AI\\data\\Sam_in_colab\\SAM_experiment.tif'

Display the downloaded image on the map.

In [20]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(center=[24.7012055, 46.6729665], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## Initialize SAM class

Set `automatic=False` to enable the `SAM2ImagePredictor`.

In [5]:
sam = SamGeo2(
    model_id="sam2-hiera-large",
    automatic=False,
)

Specify the image to segment. 

In [6]:
sam.set_image(image)

## Use an existing vector dataset as points prompts

Alternatively, you can specify a file path or HTTP URL to a vector dataset containing point geometries.

In [7]:
geojson = r'D:\Git\Geo_AI\data\Sam_in_colab\Points.geojson'

In [8]:
geojson

'D:\\Git\\Geo_AI\\data\\Sam_in_colab\\Points.geojson'

In [26]:
geojson

'd:\\Git\\Geo_AI\\data\\Point_fo_Sam.geojson'

Display the vector data on the map.

In [30]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
m.add_circle_markers_from_xy(
    geojson, radius=3, color="red", fill_color="yellow", fill_opacity=0.8
)
m

Map(center=[24.7012055, 46.6729665], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

## Segment image with a vector dataset

Segment the image using the specified file path to the vector dataset.

In [9]:
output_masks = "building_masks.tif"

In [10]:
sam.predict_by_points(
    point_coords_batch=geojson,
    point_crs="EPSG:4326",
    output=output_masks,
    dtype="uint8",
    multimask_output=False,
)

Display the segmented masks on the map.

In [ ]:
m.add_raster(
    output_masks, cmap="jet", nodata=0, opacity=0.7, layer_name="Building masks"
)
m

![image](https://github.com/user-attachments/assets/262e1a31-1648-47d2-9e71-c85ab15b1a5c)

## Clean up the result

Remove small objects from the segmented masks, fill holes, and compute geometric properties.

In [11]:
out_vector = "building_vector.geojson"
out_image = "buildings.tif"

In [12]:
array, gdf = sam.region_groups(
    output_masks, min_size=200, out_vector=out_vector, out_image=out_image
)

In [ ]:
gdf.head()

![image](https://github.com/user-attachments/assets/af9ffa11-8ebe-4b42-8cba-3f5bcc4912f4)

## Regularize building footprints

Regularize the building footprints using the `regularize()` method.

In [13]:
output_regularized = "building_regularized.geojson"
regularize(out_vector, output_regularized)

Display the regularized building footprints on the map.

In [ ]:
m = leafmap.Map()
m.add_raster(image, layer_name="Image")
style = {
    "color": "#ffff00",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0,
}
m.add_raster(out_image, cmap="tab20", opacity=0.7, nodata=0, layer_name="Buildings")
m.add_vector(
    output_regularized, style=style, layer_name="Building regularized", info_mode=None
)
m

![image](https://github.com/user-attachments/assets/b39ee029-2089-45b8-8ac0-ba0d750cec22)

## Interactive segmentation

In [ ]:
sam.show_map()

![](https://github.com/user-attachments/assets/4f487505-6e89-4892-9a70-95ab0aa69cb6)